In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd

%matplotlib inline

In [7]:
cap = cv.VideoCapture('data/Video1_husky.mp4')

background_remover = cv.createBackgroundSubtractorKNN(detectShadows=True)

# Read until video is completed and store the frames
frames = []
max_cnts = []

while(cap.isOpened()):
    ret, frame = cap.read()

    #apply gaussian blur to reduce noise
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    
    mask = background_remover.apply(frame)
    _, mask = cv.threshold(mask, 254, 255, cv.THRESH_BINARY)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((3,3),np.uint8))

    contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    max_cnt = [0, 0, 0]
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area > 800 and len(cnt) >= 5:
            if area > max_cnt[0]:
                max_cnt = [area, cnt]
    
    if max_cnt[0] != 0:
        elipse = cv.fitEllipse(max_cnt[1])
        cv.ellipse(frame, elipse, (0, 255, 0), 2)
    #cv.drawContours(frame, [cnt], -1, (0, 255, 0), 2)


    cv.imshow('mask', frame)

    if ret == True:
        frames.append(frame)
        max_cnts.append(max_cnt)

    key = cv.waitKey(33)
    
    if key == 27:
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
# convert frames to numpy array and rgb
frames = [cv.cvtColor(frame, cv.COLOR_BGR2RGB) for frame in frames]
frames = np.array(frames)

In [14]:
background_remover = cv.createBackgroundSubtractorKNN(detectShadows=False)
mask = [background_remover.apply(frame) for frame in frames]